<a href="https://colab.research.google.com/github/Airujing/Super_Resolution/blob/main/SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pwd

/content


In [3]:
%cd drive/MyDrive/SwinIR_SR
!pwd
%ll

/content/drive/MyDrive/SwinIR_SR
/content/drive/MyDrive/SwinIR_SR
total 298
drwx------ 2 root  4096 Dec  5 06:50 data/
drwx------ 2 root  4096 Dec  5 12:24 DIV2K/
drwx------ 2 root  4096 Dec  5 06:50 docs/
drwx------ 2 root  4096 Dec  5 06:50 figs/
drwx------ 2 root  4096 Dec  5 06:50 kernels/
-rw------- 1 root  1066 Nov  7 05:57 LICENSE
-rw------- 1 root  6688 Nov  7 05:57 main_challenge_sr.py
-rw------- 1 root  8602 Nov  7 05:57 main_download_pretrained_models.py
-rw------- 1 root  4461 Nov  7 05:57 main_test_dncnn3_deblocking.py
-rw------- 1 root  7640 Nov  7 05:57 main_test_dncnn.py
-rw------- 1 root  8165 Nov  7 05:57 main_test_dpsr.py
-rw------- 1 root  7251 Nov  7 05:57 main_test_face_enhancement.py
-rw------- 1 root  7274 Nov  7 05:57 main_test_fdncnn.py
-rw------- 1 root  6995 Nov  7 05:57 main_test_ffdnet.py
-rw------- 1 root  7953 Nov  7 05:57 main_test_imdn.py
-rw------- 1 root  6646 Nov  7 05:57 main_test_ircnn_denoiser.py
-rw------- 1 root  8203 Nov  7 05:57 main_test_msr

In [ ]:
!wget https://cv.snu.ac.kr/research/EDSR/DIV2K.tar

--2024-12-05 12:05:35--  https://cv.snu.ac.kr/research/EDSR/DIV2K.tar
Resolving cv.snu.ac.kr (cv.snu.ac.kr)... 147.46.67.83
Connecting to cv.snu.ac.kr (cv.snu.ac.kr)|147.46.67.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7648665600 (7.1G) [application/x-tar]
Saving to: ‘DIV2K.tar’

DIV2K.tar           100%[===================>]   7.12G  3.89MB/s    in 18m 12s 

2024-12-05 12:23:48 (6.68 MB/s) - ‘DIV2K.tar’ saved [7648665600/7648665600]



In [ ]:
!tar -xvf DIV2K.tar -C DIV2K

In [ ]:
!python main_train_psnr.py --opt options/swinir/train_swinir_denoising_gray_CN.json


In [4]:
!python main_test_swinir.py --task real_sr --scale 2 --model_path model_zoo/swinir/SR/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN.pth --folder_lq DIV2K/DIV2K/DIV2K_test_LR_unknown/X2


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
loading model from model_zoo/swinir/SR/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN.pth
/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/drive/MyDrive/SwinIR_SR/main_test_swinir.py:178: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.

In [9]:
import cv2
import os
import numpy as np
def concatenate_images(original_folder, super_res_folder, output_folder):
    original_images = load_images_from_folder(original_folder, '.png')[:12]
    super_res_images = load_images_from_folder(super_res_folder, '.bmp')[:12]

    print(f"找到 {len(original_images)} 张原始图像和 {len(super_res_images)} 张超分图像。")

    for (orig_name, orig_img), (super_res_name, super_res_img) in zip(original_images, super_res_images):
        print(f"处理图像: {orig_name} 和 {super_res_name}")
        if orig_name[:-4] == super_res_name[:-4]:  # 比较文件名去掉后缀
            # 添加标注
            orig_img_annotated = annotate_image(orig_img.copy(), "Original")
            super_res_img_annotated = annotate_image(super_res_img.copy(), "Super Res")

            # 拼接四列
            combined_img = np.hstack((orig_img_annotated, super_res_img_annotated, orig_img_annotated, super_res_img_annotated))
            output_path = os.path.join(output_folder, orig_name)
            cv2.imwrite(output_path, combined_img)
            print(f"生成拼接图像: {output_path}")
        else:
            print(f"文件名不匹配: {orig_name} 和 {super_res_name}")

# 运行代码
concatenate_images(original_folder, super_res_folder, output_folder)


def load_images_from_folder(folder, file_extension):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(file_extension):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append((filename, img))
    return images

def annotate_image(image, text):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, text, (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return image

def concatenate_images(original_folder, super_res_folder, output_folder):
    original_images = load_images_from_folder(original_folder, '.png')[:12]
    super_res_images = load_images_from_folder(super_res_folder, '.bmp')[:12]

    for (orig_name, orig_img), (super_res_name, super_res_img) in zip(original_images, super_res_images):
        if orig_name[:-4] == super_res_name[:-4]:  # 比较文件名去掉后缀
            # 添加标注
            orig_img_annotated = annotate_image(orig_img.copy(), "Original")
            super_res_img_annotated = annotate_image(super_res_img.copy(), "Super Res")

            # 拼接四列
            combined_img = np.hstack((orig_img_annotated, super_res_img_annotated, orig_img_annotated, super_res_img_annotated))
            output_path = os.path.join(output_folder, orig_name)
            cv2.imwrite(output_path, combined_img)

# 设置文件夹路径
original_folder = 'DIV2K/DIV2K/DIV2K_test_LR_unknown/X2'  # PNG 文件夹路径
super_res_folder = 'results/swinir_real_sr_x2'  # BMP 文件夹路径
output_folder = 'results'  # 输出文件夹路径

# 创建输出文件夹（如果不存在）
os.makedirs(output_folder, exist_ok=True)

# 执行拼接
concatenate_images(original_folder, super_res_folder, output_folder)


找到 12 张原始图像和 12 张超分图像。
处理图像: 0903x2.png 和 0901x2.bmp
文件名不匹配: 0903x2.png 和 0901x2.bmp
处理图像: 0907x2.png 和 0902x2.bmp
文件名不匹配: 0907x2.png 和 0902x2.bmp
处理图像: 0910x2.png 和 0903x2.bmp
文件名不匹配: 0910x2.png 和 0903x2.bmp
处理图像: 0906x2.png 和 0904x2.bmp
文件名不匹配: 0906x2.png 和 0904x2.bmp
处理图像: 0902x2.png 和 0905x2.bmp
文件名不匹配: 0902x2.png 和 0905x2.bmp
处理图像: 0911x2.png 和 0906x2.bmp
文件名不匹配: 0911x2.png 和 0906x2.bmp
处理图像: 0904x2.png 和 0907x2.bmp
文件名不匹配: 0904x2.png 和 0907x2.bmp
处理图像: 0901x2.png 和 0908x2.bmp
文件名不匹配: 0901x2.png 和 0908x2.bmp
处理图像: 0912x2.png 和 0909x2.bmp
文件名不匹配: 0912x2.png 和 0909x2.bmp
处理图像: 0909x2.png 和 0910x2.bmp
文件名不匹配: 0909x2.png 和 0910x2.bmp
处理图像: 0908x2.png 和 0911x2.bmp
文件名不匹配: 0908x2.png 和 0911x2.bmp
处理图像: 0905x2.png 和 0912x2.bmp
文件名不匹配: 0905x2.png 和 0912x2.bmp


In [15]:
import cv2
import os
import numpy as np

def load_images_from_folder(folder, file_extension):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(file_extension):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append((filename, img))
    return images

def annotate_image(image, text):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, text, (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return image

def resize_images(orig_img, super_res_img):
    # 调整超分图像的大小以匹配原始图像的宽度和高度
    height, width = orig_img.shape[:2]
    super_res_resized = cv2.resize(super_res_img, (width, height))
    return super_res_resized

def concatenate_images(original_folder, super_res_folder, output_folder):
    original_images = load_images_from_folder(original_folder, '.png')[:12]
    super_res_images = load_images_from_folder(super_res_folder, '.bmp')[:12]

    print(f"找到 {len(original_images)} 张原始图像和 {len(super_res_images)} 张超分图像。")

    combined_rows = []

    for orig_name, orig_img in original_images:
        orig_prefix = orig_name[:-4]
        matched = False

        for super_res_name, super_res_img in super_res_images:
            super_res_prefix = super_res_name[:-4]

            if orig_prefix in super_res_prefix:
                matched = True

                # 调整超分图像大小
                super_res_img_resized = resize_images(orig_img, super_res_img)

                # 添加标注
                orig_img_annotated = annotate_image(orig_img.copy(), "Original")
                super_res_img_annotated = annotate_image(super_res_img_resized.copy(), "Super Res")

                # 将原图和超分图像放入同一行
                combined_row = np.hstack((orig_img_annotated, super_res_img_annotated))
                combined_rows.append(combined_row)
                break

        if not matched:
            print(f"未找到匹配的超分图像: {orig_name}")

    # 确保有足够的行以形成 6 行 4 列
    while len(combined_rows) < 6:
        # 创建一个与已存在行相同大小的空白行
        if combined_rows:
            empty_row = np.zeros_like(combined_rows[0])
        else:
            empty_row = np.zeros((500, 2040, 3), dtype=np.uint8)  # 500 是高度，2040 是宽度，可根据需要调整
        combined_rows.append(empty_row)

    # 调整每一行的宽度一致
    max_width = max(row.shape[1] for row in combined_rows)
    for i in range(len(combined_rows)):
        if combined_rows[i].shape[1] < max_width:
            pad_width = max_width - combined_rows[i].shape[1]
            padding = np.zeros((combined_rows[i].shape[0], pad_width, 3), dtype=np.uint8)
            combined_rows[i] = np.hstack((combined_rows[i], padding))

    # 最终拼接成 6 行
    combined_image = np.vstack(combined_rows)

    output_path = os.path.join(output_folder, 'combined_image.png')
    cv2.imwrite(output_path, combined_image)
    print(f"生成拼接图像: {output_path}")


# 设置文件夹路径
original_folder = 'DIV2K/DIV2K/DIV2K_test_LR_unknown/X2'  # PNG 文件夹路径
super_res_folder = 'results/swinir_real_sr_x2'  # BMP 文件夹路径
output_folder = 'results'  # 输出文件夹路径

# 创建输出文件夹（如果不存在）
os.makedirs(output_folder, exist_ok=True)

# 执行拼接
concatenate_images(original_folder, super_res_folder, output_folder)


找到 12 张原始图像和 12 张超分图像。
生成拼接图像: results/combined_image.png


In [16]:
!git clone https://github.com/Airujing/Super_Resolution.git

Cloning into 'Super_Resolution'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.
